In [9]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('fast')

from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Embedding, concatenate
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split


from sklearn.preprocessing import OneHotEncoder

In [10]:
df_completo = pd.read_csv('df_merged.csv')

In [11]:
#Convertimos variables en el tipo correspondiente

df_completo['ID_tienda'] = df_completo['ID_tienda'].astype(object)
df_completo['Fecha_venta'] = pd.to_datetime(df_completo['Fecha_venta'])
df_completo['ID_producto'] = df_completo['ID_producto'].astype(object)
df_completo['Status'] = df_completo['Status'].astype(object)
df_completo['ID_proveedor'] = df_completo['ID_proveedor'].astype(object)
df_completo['Categoria'] = df_completo['Categoria'].astype(object)
df_completo['Clasif_proveedor'] = df_completo['Clasif_proveedor'].astype(object)
df_completo['Año'] = df_completo['Año'].astype(object)
df_completo['Mes'] = df_completo['Mes'].astype(object)
df_completo['Dia_del_Mes'] = df_completo['Dia_del_Mes'].astype(object)
df_completo['Cluster'] = df_completo['Cluster'].astype(object)

In [12]:
"""
# Filtrar datos para cada cluster
cluster_0 = df_completo[df_completo['Cluster'] == 0]
cluster_1 = df_completo[df_completo['Cluster'] == 1]
cluster_2 = df_completo[df_completo['Cluster'] == 2]
cluster_3 = df_completo[df_completo['Cluster'] == 3]

SyntaxError: incomplete input (2352570329.py, line 1)

In [13]:
# Eliminar las columnas 'Lag_1', 'Rolling_Mean_7' y 'Rolling_Std_7'
df_completo = df_completo.drop(columns=['Lag_1', 'Rolling_Mean_7', 'Rolling_Std_7', 'Cluster', 'ID_tienda', 'Facturacion', 'Status', 'ID_proveedor', 'Categoria', 'Clasif_proveedor'])

In [14]:
# Cargar y preparar los datos
df_completo['Fecha_venta'] = pd.to_datetime(df_completo['Fecha_venta'])
df_completo.set_index('Fecha_venta', inplace=True)

In [15]:
df_completo.to_csv('df_completo.csv', index=True)

In [16]:
# Separar características numéricas y categóricas
numeric_data = df_completo[['Precio_unidad']]
categorical_data = df_completo[['ID_producto', 'Categoria', 'Promocion', 'Dia_sem', 'Mes', 'Dia_del_Mes']]

KeyError: "['Categoria'] not in index"

In [ ]:
# Normalizar características numéricas
scaler = MinMaxScaler()
numeric_data = scaler.fit_transform(numeric_data)

In [ ]:
# Codificar características categóricas
encoder = OneHotEncoder()  # Asegúrate de tener la última versión de sklearn o elimina el parámetro sparse
categorical_data_encoded = encoder.fit_transform(categorical_data)
#numeric_data = encoder.fit_transform(numeric_data)

In [ ]:
print(numeric_data.shape)
print(categorical_data_encoded.shape)


In [ ]:
categorical_data_encoded

In [ ]:
numeric_data

In [ ]:
categorical_data_encoded = categorical_data_encoded.toarray()

In [ ]:
# Simulación basada en tus dimensiones
#numeric_data = np.random.rand(113254, 1)  # Datos numéricos aleatorios
#categorical_data_encoded = np.random.rand(113254, 982)  # Datos categóricos codificados aleatorios

# Concatenar datos numéricos y categóricos
data = np.hstack((numeric_data, categorical_data_encoded))

In [ ]:
import tensorflow as tf

# Verificar dispositivos disponibles, GPU debería aparecer si está configurado correctamente
print("Dispositivos disponibles:", tf.config.list_physical_devices())

# Configurar TensorFlow para que use la GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configuración para evitar el uso de toda la memoria de la GPU de una sola vez
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [ ]:
import cupy as cp
# Parámetros para la preparación de datos
n_future = 15  # Número de días que deseamos prever en el futuro
n_past = 1096  # Número de días pasados que usaremos para predecir el futuro

# Asegúrate de que 'data' sea un array de CuPy
data2 = cp.asarray(data)

# Preparar datos para LSTM usando CuPy
X, y = [], []
for i in range(n_past, len(data) - n_future + 1):
    X.append(data2[i - n_past:i])
    y.append(data2[i + n_future - 1:i + n_future, 0])

# Convertir listas a arrays de CuPy
X = cp.array(X)
y = cp.array(y)

In [ ]:
"""
# Preparar datos para LSTM
n_future = 15  # Número de días que deseamos prever en el futuro
n_past = 1096   # Número de días pasados que usaremos para predecir el futuro
X, y = [], []
for i in range(n_past, len(data) - n_future + 1):
    X.append(data[i - n_past:i])
    y.append(data[i + n_future - 1:i + n_future, 0])  # Asumiendo que el objetivo es la primera columna

X, y = np.array(X), np.array(y)

In [ ]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Definir el modelo LSTM
input_shape = (X_train.shape[1], X_train.shape[2])
model_input = Input(shape=input_shape)
x = LSTM(50, return_sequences=True)(model_input)
x = Dropout(0.2)(x)
x = LSTM(50)(x)
x = Dense(1)(x)  # Salida

In [ ]:
model = Model(inputs=model_input, outputs=x)
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
# Evaluar el modelo
predictions = model.predict(X_test)

In [ ]:
predictions

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Simulación de datos para el ejemplo
# 'y_test' serían tus valores reales y 'predictions' las predicciones del modelo
y_test = np.random.rand(100)  # 100 valores reales simulados
predictions = y_test + np.random.normal(0, 0.1, 100)  # Predicciones con ruido añadido


In [ ]:
# Crear gráfico para comparar predicciones con la realidad
plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Realidad', color='blue')
plt.plot(predictions, label='Predicciones', color='red', linestyle='--')
plt.title('Comparación de Predicciones vs Realidad')
plt.xlabel('Tiempo / Índice')
plt.ylabel('Valor de la Serie Temporal')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool, cpu_count


In [ ]:
# Función para predecir para un producto usando el modelo de red neuronal
def predecir_producto_neural(args):
    producto, fecha_inicio = args
    dias_a_predecir = 15
    fechas_prediccion = pd.date_range(start=fecha_inicio, periods=dias_a_predecir, freq='D')
    df_prediccion = pd.DataFrame(fechas_prediccion, columns=['Fecha_venta'])
    # resto del código...

    df_prediccion['ID_producto'] = producto
    df_prediccion['Precio_unidad'] = df_prediccion['Fecha_venta'].apply(lambda x: obtener_precio(x, producto))

    # Agregar más transformaciones o adaptaciones según las necesidades del modelo neural
    # Aquí es donde necesitas asegurarte de que los datos están en el formato correcto para el modelo
    # Por ejemplo, si tu modelo toma entradas normalizadas o tiene requerimientos específicos de forma

    # Preparar datos para la red neuronal
    # Esto incluiría normalización, y tal vez reshaping si es necesario para el modelo
    input_data = preparar_datos_para_red_neural(df_prediccion)

    # Realizar predicciones con el modelo de red neuronal
    predicciones = neural_network_model.predict(input_data)

    # Post-procesar las predicciones si es necesario (por ejemplo, redondeo, ajuste de escala)
    df_prediccion['Predicciones'] = predicciones.flatten()  # Asegúrate de ajustar esta línea según sea necesario

    return df_prediccion

In [ ]:
# Normalización de los datos numéricos
scaler = StandardScaler()

def normalizar_columnas(df, columnas):
    df[columnas] = scaler.fit_transform(df[columnas])
    return df

# Suponiendo que df_merged es tu DataFrame completo con datos de todos los productos
df_completo = normalizar_columnas(df_completo, ['Cant_vendidas', 'Precio_unidad'])

In [ ]:
# Función para predecir para un producto usando el modelo de red neuronal
def predecir_producto_neural(args):
    producto, fecha_inicio = args
    dias_a_predecir = 15
    fechas_prediccion = pd.date_range(start=fecha_inicio, periods=dias_a_predecir, freq='D')
    df_prediccion = pd.DataFrame(fechas_prediccion, columns=['Fecha_venta'])
    # resto del código...

    df_prediccion['ID_producto'] = producto
    df_prediccion['Precio_unidad'] = df_prediccion['Fecha_venta'].apply(lambda x: obtener_precio(x, producto))

    # Agregar más transformaciones o adaptaciones según las necesidades del modelo neural
    # Aquí es donde necesitas asegurarte de que los datos están en el formato correcto para el modelo
    # Por ejemplo, si tu modelo toma entradas normalizadas o tiene requerimientos específicos de forma

    # Preparar datos para la red neuronal
    # Esto incluiría normalización, y tal vez reshaping si es necesario para el modelo
    input_data = preparar_datos_para_red_neural(df_prediccion)

    # Realizar predicciones con el modelo de red neuronal
    predicciones = neural_network_model.predict(input_data)

    # Post-procesar las predicciones si es necesario (por ejemplo, redondeo, ajuste de escala)
    df_prediccion['Predicciones'] = predicciones.flatten()  # Asegúrate de ajustar esta línea según sea necesario

    return df_prediccion

In [ ]:
def obtener_precio(fecha, producto):
    # Supongamos que df_merged_last_prices es un DataFrame que tiene precios históricos de productos
    # Intenta obtener el precio en la fecha dada. Si no es posible, devuelve el precio medio conocido.
    precio = df_merged_last_prices[(df_merged_last_prices['ID_producto'] == producto) &
                                   (df_merged_last_prices['Fecha_venta'] <= fecha)].nlargest(1, 'Fecha_venta')['Precio_unidad']
    if precio.empty:
        return df_merged_last_prices[df_merged_last_prices['ID_producto'] == producto]['Precio_unidad'].mean()
    else:
        return precio.iloc[0]


In [ ]:
# df_completo['Precio_unidad'] = df_completo['Fecha_venta'].apply(lambda x: obtener_precio(x, producto))


In [ ]:
# Asegúrate de que 'cluster_0' está definido correctamente y contiene la columna 'ID_producto'
productos = df_completo['ID_producto'].unique()
# Ejecución en paralelo para todos los productos (si es necesario)
if __name__ == "__main__":
    # Convertimos la fecha de inicio a un objeto datetime
    fecha_inicio = datetime.strptime('2024-07-01', "%Y-%m-%d")  # Asegúrate de obtener esta fecha correctamente
    args = [(producto, fecha_inicio) for producto in productos]

    with Pool(cpu_count()) as pool:
        all_predictions = pool.map(predecir_producto_neural, args)

    # Combinar todas las predicciones en un solo DataFrame
    all_predictions_df = pd.concat(all_predictions, ignore_index=True)

    # Procesar y guardar los resultados como lo hiciste anteriormente
    all_predictions_df.to_csv('predicciones_neural_network.csv', index=False)